ФИО: Холкин Николай Алексеевич

# Задание 1

Задача: Создать чат бота для получения информации об исследованиях космоса

Описание: Создайте комплексное приложение командной строки, которое будет использоваться в качестве панели управления исследованиями космоса. Данное приложение будет обращаться к https://api.nasa.gov/ для предоставления пользователям набора информации о космосе, включая:

- Астрономическая картинка дня (APOD): Отображение APOD с пояснениями к нему.
- Фотографии с марсохода: позволяет пользователям выбирать и фильтровать фотографии с марсохода по дате и типу камеры.
- Объекты, сближающиеся с Землей (ОСЗ): Поиск и отображение информации об объекте, сближающихся с Землей, на определенную дату, включая их размеры и потенциальную опасность.
- Данные о космической погоде: Отображают последние данные о космической погоде, включая солнечные вспышки и геомагнитные бури.
Приложение должно позволять пользователям ориентироваться в этих функциях, корректно обрабатывать ошибки и обеспечивать удобство работы.

Требования:
- Пользовательский ввод: Приложение должно предложить пользователю ввести данные, чтобы выбрать, какую функцию он хочет изучить.
- Проверка данных: Убедитесь, что пользовательские данные (например, даты) проверены.
- Обработка ошибок: Корректно обрабатывайте ошибки API и неверные ответы.
- Представление данных: Представляйте данные в четкой и организованной форме.
- Опция выхода: позволяет пользователям выходить из приложения в любое время.

In [1]:
!pip install python-dotenv
from dotenv import load_dotenv
from os import getenv

from typing import TypedDict
import datetime as dt
import requests

class APOD(TypedDict):
    """Неполное представление значения, возвращаемого APOD API."""
    title: str
    date: str
    url: str
    explanation: str
    copyright: str


def validate_user_date(user_date: str) -> bool:
    """
    Проверяет соответствие введенной пользователем даты на соответствие формату
    YYYY-MM-DD.
    :param user_date: Дата пользователя
    """
    return True  # FIXME!! DEBUG ONLY!


def print_apod(apod_info: APOD) -> None:
    """Выводит в stdout данные об APOD."""
    print(f'Название фото: {apod_info["title"]}')
    print(f'Ссылка на фото: {apod_info["url"]}')
    explanation = apod_info["explanation"].replace('  ', '\n')
    print(f'Подпись к фото:\n{explanation}')
    print(f'Авторские права: {apod_info["copyright"]}')


def print_asteroids(cur_date: str, response: dict) -> None:
    """
    Выводит в консоль информацию об астероидах, которые проходят рядом с Землей
    в конкретный день, полученный от NASA NeoWs API
    :param cur_date: Сегодняшняя дата формата YYYY-mm-dd
    """
    for asteroid in jsoned_response['near_earth_objects'][cur_date]:
        min_diam_meters, max_diam_meters = asteroid['estimated_diameter']['meters'].values()
        hazard = asteroid['is_potentially_hazardous_asteroid']

        print(f'Мин. диаметр: {min_diam_meters:.2f} метров',
              f'Макс. диаметр: {max_diam_meters:.2f} метров',
              f'Опасен ли астероид: {"Да" if hazard else "Нет"}', sep='\n')


load_dotenv('venv.env')
TOKEN_API = getenv('NASA_API')

GREETING_MSG = 'Вы авторизовались в приложении SpaceInfoBot. Список операций:'
OPERATIONS_LIST = ('Покажи астрономическую картину дня (APOD)',
                   'Покажи фотографии с марсохода',
                   'Покажи объекты, сближающиеся с Землей',
                   'Выведи данные о космической погоде',
                   'Выйти из приложения')
ROVERS_LIST = {'0': 'curiosity',
               '1': 'opportunity',
               '2': 'spirit'}
ROVER_CAMERAS_LIST = {'0': 'fhaz',
                      '1': 'rhaz',
                      '2': 'navcam'}

operations = tuple(f'{num}. {action}' for num, action in enumerate(OPERATIONS_LIST))
rovers = tuple(f'{num}. {rover}' for num, rover in ROVERS_LIST.items())
cameras = tuple(f'{num}. {camera}' for num, camera in ROVER_CAMERAS_LIST.items())
cur_date = dt.datetime.utcnow().strftime('%Y-%m-%d')

print(GREETING_MSG)
print(*operations, sep='\n')

while True:
    op_num = input('Введите номер операции (без точки): ')
    if op_num == '0':  # APOD
        response = requests.get(f'https://api.nasa.gov/planetary/apod?api_key={TOKEN_API}')
        if response.status_code in (200, 201):
            jsoned_response = response.json()
            print_apod(jsoned_response)

    elif op_num == '1':  # Фото марсохода
        print('Напишите номер марсохода (без точки):', *rovers, sep='\n')
        rover_name = ROVERS_LIST.get(input())

        if rover_name is not None:  # rover_name is valid
            date_photo = input('Введите дату (формат YYYY-MM-DD): ')
            if validate_user_date(date_photo): # date is valid
                print('Введите номер камеры', *cameras, sep='\n')
                camera = ROVER_CAMERAS_LIST.get(input())
                if camera is not None:  # camera is valid
                    response = requests.get(f'https://api.nasa.gov/mars-photos/api/v1/rovers/{rover_name}/photos?earth_date={date_photo}&camera={camera}&api_key={TOKEN_API}')
                    if response.status_code in (200, 201):
                        photos = [photo['img_src'] for photo in response.json()['photos']]
                        print(*photos, sep='\n')
                    else:
                        print('Запрос сформирован неверно')
                else:
                    print('Вы ввели неверный номер камеры')
            else:
                print('Вы ввели неверную дату')
        else:
            print('Марсохода с таким номером нет')
    elif op_num == '2':  # ОСЗ
        response = requests.get(f'https://api.nasa.gov/neo/rest/v1/feed?start_date={cur_date}&end_date={cur_date}&api_key={TOKEN_API}')
        if response.status_code in (200, 201):
            jsoned_response = response.json()
            print_asteroids(cur_date, jsoned_response)
        else:
            print(f'Ответ сервера неверный. Код ответа - {response.status_code}')

    elif op_num == '3':  # Космическая погода
        gst_response = requests.get(f'https://api.nasa.gov/DONKI/GST?startDate={cur_date}&endDate={cur_date}&api_key={TOKEN_API}')
        flr_response = requests.get(f'https://api.nasa.gov/DONKI/FLR?startDate={cur_date}&endDate={cur_date}&api_key={TOKEN_API}')
        if (gst_response.status_code in (200, 201) and
            flr_response.status_code in (200, 201)):
            geomagnetic_storms = [(gst['observedTime'], gst['kpIndex']) \
                                  for geom_st in gst_response.json()
                                  for gst in geom_st['allKpIndex']]

            sun_flares = [(flr['beginTime'], flr['classType']) \
                          for flr in flr_response.json()]

            if geomagnetic_storms:
                print('Геомагнитные бури:')
                for time, kp_index in geomagnetic_storms:
                    time = dt.datetime.fromisoformat(time[:-1]).astimezone(dt.timezone.utc)
                    print(f'Время начала бури: {time}\nК-индекс: {kp_index}\n')
            else:
                print('Геомагнитных бурь сегодня нет.')

            if sun_flares:
                print('Солнечные вспышки:')
                for date, class_type in sun_flares:
                    date = dt.datetime.fromisoformat(date[:-1]).astimezone(dt.timezone.utc).strftime('%Y-%m-%d %H:%M')
                    print(f'Дата начала вспышки: {date}\nТип вспышки: {class_type}\n')
            else:
                print('Солнечных вспышек сегодня нет.')

    elif op_num == '4':  # Ctrl-C
        break

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


C:\Users\1\AppData\Local\Temp\ipykernel_2844\3111433618.py:70: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  cur_date = dt.datetime.utcnow().strftime('%Y-%m-%d')


Вы авторизовались в приложении SpaceInfoBot. Список операций:
0. Покажи астрономическую картину дня (APOD)
1. Покажи фотографии с марсохода
2. Покажи объекты, сближающиеся с Землей
3. Выведи данные о космической погоде
4. Выйти из приложения


Введите номер операции (без точки):  0


Название фото: The Medusa Nebula
Ссылка на фото: https://apod.nasa.gov/apod/image/2411/MEDUSA_NEBULA_FINAL_BRS_SIGNED1024.jpg
Подпись к фото:
Braided and serpentine filaments of glowing gas suggest this nebula's popular name, The Medusa Nebula. Also known as Abell 21, this Medusa is an old planetary nebula some 1,500 light-years away in the constellation Gemini. Like its mythological namesake, the nebula is associated with a dramatic transformation. The planetary nebula phase represents a final stage in the evolution of low mass stars like the sun as they transform themselves from red giants to hot white dwarf stars and in the process shrug off their outer layers. Ultraviolet radiation from the hot star powers the nebular glow. The Medusa's transforming star is the faint one near the center of the overall bright crescent shape. In this deep telescopic view, fainter filaments clearly extend below and to the left. The Medusa Nebula is estimated to be over 4 light-years across.
Авторские 

Введите номер операции (без точки):  4


# Задание 2

Описание задачи

Цель этой задачи - создать скрипт на Python, который взаимодействует с API Чикагского института искусств (https://api.artic.edu/docs/) для извлечения и отображения произведений искусства. Скрипт должен позволять пользователям просматривать работы по страницам, фильтровать их по имени художника и просматривать подробную информацию о выбранных произведениях искусства. Ниже приведены требования и функциональные возможности, которые необходимо реализовать:

Требования:
Извлекать произведения искусства:

- Создайте функцию, которая извлекает список произведений искусства из API Чикагского института искусств.
Функция должна принимать параметр page для разбивки на страницы и возвращать список произведений искусства вместе с информацией о разбивке на страницы.
Фильтровать произведения искусства:

- Реализуйте функцию, которая фильтрует список произведений искусства на основе имени указанного художника. Функция должна возвращать список работ, которые соответствуют имени художника (без учета регистра).
Отображать подробную информацию об оформлении:

- Напишите функцию, которая отображает названия работ для пользователя и позволяет ему выбрать одну из них, введя соответствующий номер.
После выбора функция должна отображать подробную информацию о выбранном произведении, включая название, исполнителя, дату и носитель.
Разбивка на страницы и взаимодействие с пользователем:

- Создайте основную функцию, которая управляет выборкой произведений и взаимодействием с пользователем.

Разрешите пользователям перемещаться по страницам с произведениями искусства, выполнять фильтрацию по исполнителю или выходить из программы.

Если страниц с произведениями искусства несколько, укажите варианты перехода к следующей странице, предыдущей странице, фильтрации по исполнителю или выхода из программы.

In [ ]:
import requests as r
from typing import TypedDict
from sys import stderr

class Pagination(TypedDict):
    """Представление значений по ключу pagination из ответов api.artic.edu"""
    total: int
    limit: int
    offset: int
    total_pages: int
    current_page: int
    next_url: str


class Artwork(TypedDict):
    """
    Неполное представление элемента списка data
    из ответа https://api.artic.edu/api/v1/artworks
    """
    id: int
    api_link: str
    title: str
    artist_title: str


class Artworks(TypedDict):
    """
    Представление JSON-ответа сервиса https://api.artic.edu/api/v1/artworks
    """
    pagination: Pagination
    data: list[Artwork]


def valid_json_response(url: str) -> Artworks:
    """
    Возвращает JSON-like ответ от сервиса с адресом url.
    :param url: URL сервиса
    """
    response = r.get(url)
    if response.status_code in (200, 201):
        return response.json()
    stderr.write(f'API не вернул данные. Код ответа - {response.status_code}')
    return Artworks  # костыль x_x


def print_artworks(artworks: Artworks, page: int, total_pages: int) -> str:
    """
    Выводит в консоль информацию о произведениях искусства.
    :param artworks: Экземпляр класса Artworks
    :param page: Номер текущей страницы в CLI
    :param total_pages: Общее число страниц в CLI
    """
    print(f'Страница {page}/{total_pages}')
    for artwork in artworks['data']:
        print(f'ID работы: {artwork["id"]}',
              f'Название работы: {artwork["title"]}',
              f'Имя автора: {artwork["artist_title"]}', sep='\n', end='\n\n')

    next_page = input('Введите номер следующей страницы\n(Или введите 0, ' + \
                      'чтобы закрыть диалоговое окно): ')
    print('')  # для 'красоты' CLI
    return next_page


def print_searched_arts(artworks: Artworks, page: int,
                        total_pages: int) -> str:
    print(f'Страница {page}/{total_pages}')
    for searched_artwork in artworks['data']:
        artwork = searched_artwork
        print(f'ID работы: {artwork["id"]}',
              f'Название работы: {artwork["title"]}', sep='\n', end='\n\n')

    next_page = input('Введите номер следующей страницы\n(Или введите 0, ' + \
                      'чтобы закрыть диалоговое окно): ')
    print('')  # для 'красоты' CLI
    return next_page


def get_artworks_by_page(page: int = '1') -> None:
    """
    Получает страницу из списка произведений искусств в коллекции
    Чикагского института искусств.
    :param page: Номер страницы (6 элементов/страница)
    """
    json_response = valid_json_response(f'https://api.artic.edu/api/v1/artworks?page={page}&limit=6')
    total_pages = json_response['pagination']['total_pages']
    if json_response:
        while page != '0':
            page = print_artworks(json_response, page, total_pages)
    else:
        print('Возникла ошибка в ходе получения ответа от сервера.')


def get_artworks_by_author(author: str, page: int = '1') -> None:
    """
    Получает страницу из списка произведений искусств для автора author
    :param author: Имя автора
    :param page: Номер страницы (6 элементов/страница)
    """
    json_response = valid_json_response(f'https://api.artic.edu/api/v1/artworks/search?q={author}&limit=6')
    total_pages = json_response['pagination']['total_pages']
    if json_response:
        while page != '0':
            page = print_searched_arts(json_response, page, total_pages)
    else:
        print('Возникла ошибка в ходе получения ответа от сервера.')


def get_artwork_by_title(title: str, page: int = '1') -> None:
    """
    Получает страницу из списка произведений искусств для названия title
    :param title: Название картины
    :param page: Номер страницы (6 элементов/страница)
    """
    json_response = valid_json_response(f'https://api.artic.edu/api/v1/artworks/search?q={title}&limit=6')
    total_pages = json_response['pagination']['total_pages']
    if json_response:
        while page != '0':
            page = print_searched_arts(json_response, page, total_pages)
    else:
        print('Возникла ошибка в ходе получения ответа от сервера.')


OPERATIONS = ('Показать все произведения искусств из базы университета',
              'Отфильтровать произведения искусства по автору',
              'Найти информацию о произведении по названию',
              'Завершить работу скрипта')
ENUM_OPERATIONS = (f'{num}. {oper}' for num, oper in enumerate(OPERATIONS))

print('Вы авторизовались в скрипте, работающим с API AIC.',
      'Список операций:', *ENUM_OPERATIONS, sep='\n')

while True:
    oper_num = input('Введите номер операции (без точки): ')
    print('')  # для 'красоты' CLI

    if oper_num == '0':  # выбор по номеру
        get_artworks_by_page()

    elif oper_num == '1':  # фильтр по автору
        author = input('Введите имя автора:\n')
        get_artworks_by_author(author)

    elif oper_num == '2':  # поиск по названию
        title = input('Введите название картины:\n')
        get_artwork_by_title(title)

    elif oper_num == '3':  # Ctrl-C
        print('Скрипт завершил работу.')
        break

    else:
        print('Вы ввели неверный номер операции.')

Overwriting task2.py


# Задание 3

Задача: Создать программу по управлению портфелем криптовалют

Цель: Создать скрипт на Python, который извлекает цены на криптовалюты в режиме реального времени, позволяет пользователям управлять портфелем криптовалют, вычисляет общую стоимость портфеля, отслеживает изменения цен и предоставляет исторические данные о ценах для анализа.

Требования:
Получение текущих цен на криптовалюты:

Используйте https://docs.coingecko.com/ для получения актуальных цен на список криптовалют.

Управление портфелем:

- Позволяет пользователю создавать портфель криптовалют и управлять им, указывая количество каждой криптовалюты, которой он владеет.
- Расчитывает общую стоимость портфеля в указанной фиатной валюте (например, долларах США).

Отслеживание изменения цен:

- Отображение процентного изменения цены для каждой криптовалюты в портфеле за последние 24 часа.
- Выделите все криптовалюты, стоимость которых значительно увеличилась или снизилась.

Поиск исторических данных о ценах:

- Получение исторических данных о ценах на указанную криптовалюту за последнюю неделю.
- Предоставьте пользователю возможность визуализировать эти данные в простом текстовом формате (например, цены за день).

Взаимодействие с пользователем:

- Реализуйте интерфейс командной строки для ввода данных пользователем.
- Предоставьте опции для получения текущих цен, управления портфелем, просмотра изменений цен или анализа исторических данных.


# Дополнительно: Задание 4

Задание 4: Проектное

Вам необходимо самостоятельно найти откртое API предоставляющее информацию в открытом доступе и реализовать собственный проект!


Критерии приемки результата:

- Проект включает в себя не менее 5 возможостей для пользователя
- Проект позволяет использовать все возможности проекта пользователю при помощи взаимодействия через коммандную строку
- Проект работает с открытым API (это значит что при проверке вашей работы преподавателем, преподавателю необходимо просто запустить ячейку с кодом вашего проекта и она будет работать без дополнительных манипуляции)
- Проект должен обязательно включать в себя ряд используемых конструкции:
    - Функции
    - Условные конструкции
    - Ввод/вывод
    - Словари/Списки
- Допускается использование библиотек:
    - requests
    - datetime
    - random

**Здесь добавьте описание вашего проекта**

In [ ]:
#  А здесь код